# TP 4248 2021-2022 
## les membres du groupe
- fezeu Ghomsi Eugene Clotaire 17Q2864 
- Nono Mabou Wilfried Brondon 21S2817 brondonnono3@gmail.com
- NGBAYAFOU NGOUH Chéripha Cheriphalynette@gmail.com
- Junior Libii junior.libii@facsciences-uy1.cm

## binary naive bayes classification on sms dataset


In [19]:

import pandas as pd
import re
import numpy as np

dataset = pd.read_csv("./datas/spam.csv", encoding="ISO-8859-1", usecols=['v1', 'v2'], dtype={'v1':str,'v2':str})
dataset.head()
dataset = dataset.dropna(how="any")




In [22]:
!pip install nltk
import nltk
nltk.download('stopwords')
# ensemble de mots sans sens pas utilise pour la pertinence
# cette suppression offre un gain temps 
stopwords = nltk.corpus.stopwords.words('english')

#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()





     |████████████████████████████████| 1.5 MB 407 kB/s eta 0:00:01
     |████████████████████████████████| 764 kB 233 kB/s eta 0:00:01


[nltk_data] Downloading package stopwords to /home/eugene/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
def tonkenizer(text):
    return re.sub(r"[^a-zA-Z0-9]", " ", text.lower()).split()
def remove_stopwords(text):
    return[i for i in text if i not in stopwords]
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

def cleantext (text):
    return stemming(remove_stopwords(tonkenizer(text)))

dataset.v2 = dataset.v2.apply(cleantext)
dataset

AttributeError: 'list' object has no attribute 'lower'